In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

# Specify the path to the dataset directory
train_data_dir = '/content/drive/MyDrive/Dataset/Rice_Leaf_Diseases/traning'
test_data_dir = '/content/drive/MyDrive/Dataset/Rice_Leaf_Diseases/testing'

# Set the number of classes
num_classes = 3

# Set the input image size
input_size = (224, 224)

# Define the batch size
batch_size = 32

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load and augment the training data
train_set = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load and preprocess the test data
test_set = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the hybrid model
model = Model(inputs=base_model.input, outputs=predictions)

model.summary()

# Freeze the last four layers of the base model for fine-tuning
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_set, epochs=50, validation_data=test_set)

# Save the trained model
model.save('hybrid_model.h5')

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

# Set the input image size
input_size = (224, 224)

train_data_dir = '/content/drive/MyDrive/Dataset/Rice_Leaf_Diseases/traning'
batch_size = 32
# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/hybrid_model.h5')

# Specify the path to the image you want to predict
image_path = '/content/drive/MyDrive/Dataset/Rice_Leaf_Diseases/val/Bacterial leaf blight/DSC_0370.jpg'
# Load and preprocess the image
img = image.load_img(image_path, target_size=input_size)
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img / 255.0

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


train_set = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical'
)


# Make the prediction
predictions = model.predict(img)
class_labels = train_set.class_indices
predicted_class = list(class_labels.keys())[np.argmax(predictions)]

# Display the image and prediction
plt.imshow(img[0])
plt.title('Predicted class: ' + predicted_class)
plt.axis('off')
plt.show()



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

test_data_dir = '/content/drive/MyDrive/Dataset/Rice_Leaf_Diseases/testing'
num_classes = 3

# Set the input image size
input_size = (224, 224)

# Define the batch size
batch_size = 32

# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/hybrid_model.h5')

# Load and preprocess the test data
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Important: Set shuffle to False to preserve class order
)

# Make predictions on the test set
predictions = model.predict(test_set)
y_pred = np.argmax(predictions, axis=1)
y_true = test_set.classes

# Generate the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Get class labels
class_labels = list(test_set.class_indices.keys())

# Create a heatmap from the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Generate classification report
report = classification_report(y_true, y_pred, target_names=class_labels)
print(report)
